# প্রাণী বিশেষজ্ঞ সিস্টেম বাস্তবায়ন

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) থেকে একটি উদাহরণ।

এই উদাহরণে, আমরা একটি সহজ জ্ঞান-ভিত্তিক সিস্টেম বাস্তবায়ন করব যা কিছু শারীরিক বৈশিষ্ট্যের উপর ভিত্তি করে একটি প্রাণী নির্ধারণ করবে। সিস্টেমটি নিম্নলিখিত AND-OR গাছ দ্বারা উপস্থাপন করা যেতে পারে (এটি পুরো গাছের একটি অংশ, আমরা সহজেই আরও কিছু নিয়ম যোগ করতে পারি):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.bn.png)


## আমাদের নিজস্ব এক্সপার্ট সিস্টেম শেল ব্যাকওয়ার্ড ইনফারেন্স সহ

চলুন প্রোডাকশন রুলের উপর ভিত্তি করে জ্ঞান উপস্থাপনার জন্য একটি সহজ ভাষা সংজ্ঞায়িত করার চেষ্টা করি। আমরা রুল সংজ্ঞায়িত করার জন্য Python ক্লাসগুলোকে কীওয়ার্ড হিসেবে ব্যবহার করব। মূলত তিন ধরনের ক্লাস থাকবে:
* `Ask` একটি প্রশ্নকে উপস্থাপন করে যা ব্যবহারকারীকে জিজ্ঞাসা করতে হবে। এটি সম্ভাব্য উত্তরগুলোর সেট ধারণ করে।
* `If` একটি নিয়মকে উপস্থাপন করে, এবং এটি শুধুমাত্র নিয়মের বিষয়বস্তু সংরক্ষণের জন্য একটি সিনট্যাকটিক সুগার।
* `AND`/`OR` হল ক্লাসগুলো যা গাছের AND/OR শাখাগুলোকে উপস্থাপন করে। এগুলো শুধু ভিতরে আর্গুমেন্টগুলোর তালিকা সংরক্ষণ করে। কোড সহজ করার জন্য, সমস্ত কার্যকারিতা প্যারেন্ট ক্লাস `Content`-এ সংজ্ঞায়িত করা হয়েছে।


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

আমাদের সিস্টেমে, কর্মরত মেমোরি **attribute-value pairs** হিসাবে **তথ্যগুলির** তালিকা ধারণ করবে। জ্ঞানভান্ডারকে একটি বড় ডিকশনারি হিসাবে সংজ্ঞায়িত করা যেতে পারে যা ক্রিয়াগুলিকে (নতুন তথ্য যা কর্মরত মেমোরিতে যোগ করা উচিত) শর্তগুলির সাথে মানচিত্র করে, যা AND-OR অভিব্যক্তি হিসাবে প্রকাশিত হয়। এছাড়াও, কিছু তথ্য `Ask` করা যেতে পারে।


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

পিছনের দিকের অনুমান সম্পাদনের জন্য, আমরা `Knowledgebase` ক্লাসটি সংজ্ঞায়িত করব। এটি অন্তর্ভুক্ত করবে:
* কার্যকরী `memory` - একটি ডিকশনারি যা অ্যাট্রিবিউটগুলিকে মানগুলোর সাথে ম্যাপ করে
* Knowledgebase-এর `rules` - উপরে সংজ্ঞায়িত ফরম্যাটে

দুটি প্রধান পদ্ধতি হল:
* `get` - একটি অ্যাট্রিবিউটের মান প্রাপ্ত করার জন্য, প্রয়োজন হলে অনুমান সম্পাদন করে। উদাহরণস্বরূপ, `get('color')` একটি রঙের স্লটের মান পাবে (প্রয়োজন হলে জিজ্ঞাসা করবে এবং পরে ব্যবহারের জন্য কার্যকরী মেমোরিতে মানটি সংরক্ষণ করবে)। যদি আমরা `get('color:blue')` জিজ্ঞাসা করি, এটি একটি রঙ জিজ্ঞাসা করবে এবং তারপর রঙের উপর নির্ভর করে `y`/`n` মানটি ফেরত দেবে।
* `eval` - আসল অনুমান সম্পাদন করে, অর্থাৎ AND/OR গাছটি অনুসন্ধান করে, সাব-গোলগুলো মূল্যায়ন করে ইত্যাদি।


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

এখন চলুন আমাদের প্রাণী জ্ঞানভাণ্ডার সংজ্ঞায়িত করি এবং পরামর্শ কার্যক্রম সম্পাদন করি। লক্ষ্য করুন যে এই কলটি আপনাকে প্রশ্ন করবে। আপনি `y`/`n` টাইপ করে হ্যাঁ-না প্রশ্নের উত্তর দিতে পারেন, অথবা দীর্ঘ বহুবিকল্প প্রশ্নের জন্য সংখ্যা (0..N) নির্দিষ্ট করে উত্তর দিতে পারেন।


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## PyKnow ব্যবহার করে ফরোয়ার্ড ইনফারেন্স

পরবর্তী উদাহরণে, আমরা জ্ঞান উপস্থাপনার জন্য একটি লাইব্রেরি, [PyKnow](https://github.com/buguroo/pyknow/) ব্যবহার করে ফরোয়ার্ড ইনফারেন্স বাস্তবায়নের চেষ্টা করব। **PyKnow** হলো একটি লাইব্রেরি যা পাইথনে ফরোয়ার্ড ইনফারেন্স সিস্টেম তৈরি করার জন্য ব্যবহৃত হয় এবং এটি ক্লাসিক্যাল পুরনো সিস্টেম [CLIPS](http://www.clipsrules.net/index.html)-এর মতো ডিজাইন করা হয়েছে।

আমরা চাইলে নিজেরাই ফরোয়ার্ড চেইনিং বাস্তবায়ন করতে পারতাম, তেমন কোনো বড় সমস্যা ছাড়াই। তবে সাধারণ পদ্ধতিতে করা বাস্তবায়নগুলো সাধারণত খুব কার্যকর হয় না। আরও কার্যকরভাবে নিয়ম মেলানোর জন্য একটি বিশেষ অ্যালগরিদম [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) ব্যবহার করা হয়।


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

আমরা আমাদের সিস্টেমকে `KnowledgeEngine` এর সাবক্লাস হিসাবে একটি ক্লাস হিসেবে সংজ্ঞায়িত করব। প্রতিটি নিয়ম একটি পৃথক ফাংশন দ্বারা সংজ্ঞায়িত করা হয় যা `@Rule` অ্যানোটেশন ব্যবহার করে, যা নির্ধারণ করে কখন নিয়মটি কার্যকর হবে। নিয়মের ভিতরে, আমরা `declare` ফাংশন ব্যবহার করে নতুন তথ্য যোগ করতে পারি, এবং সেই তথ্য যোগ করার ফলে ফরওয়ার্ড ইনফারেন্স ইঞ্জিন দ্বারা আরও কিছু নিয়ম কার্যকর হবে।


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

একবার আমরা একটি জ্ঞানভাণ্ডার সংজ্ঞায়িত করলে, আমরা আমাদের কার্যকরী মেমোরি কিছু প্রাথমিক তথ্য দিয়ে পূরণ করি, এবং তারপর `run()` পদ্ধতি কল করি সিদ্ধান্ত গ্রহণের জন্য। এর ফলে আপনি দেখতে পাবেন যে নতুন সিদ্ধান্ত নেওয়া তথ্য কার্যকরী মেমোরিতে যোগ করা হয়েছে, যার মধ্যে প্রাণীর সম্পর্কে চূড়ান্ত তথ্যও রয়েছে (যদি আমরা সমস্ত প্রাথমিক তথ্য সঠিকভাবে সেট আপ করি)।


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
